In [14]:
import sys, os
import re
import pandas as pd
import itertools, nltk, string 

def read_file(file):
    f = open(file, 'r')

    pattern_title = '\[t\]'
    pattern_sentence = '(?<=##).+'
    pattern_aspect = '.+(?=##)'
#     pattern_aspect_anomali = '\[(u|p|s|cc|cs)\]'

    review = []
    for a in f:
        if re.search('##', a):
            sentence = re.findall(pattern_sentence, a)[0]
            aspect = re.findall(pattern_aspect, a)
#             anomali = re.findall(pattern_aspect_anomali, a)
#             if anomali:
#                 print(sentence, aspect)
#             else:
            flag = True
            if len(aspect) > 0:
                aspect = aspect[0]
                
#                 a_array = []
#                 tes = []
#                 tes = aspect.split(', ')
#                 for x in tes:
#                     a_array.append(x.split('[')[0])
#                 for a in a_array:
#                     if a not in sentence:
#                         flag = False
            else:
                aspect = ''
            if flag:
                review.append((sentence, aspect))

    df = pd.DataFrame(columns=['review','target'])
    for r in review:
        df = df.append({'review': r[0], 'target': r[1]}, ignore_index=True)
        
    return df
        
positive_lexicon = []
negative_lexicon = []

def read_lexicon():
    global positive_lexicon;
    global negative_lexicon;
    
    with open(os.path.join(os.path.abspath('opinion-lexicon-English/') , 'positive-words.txt'), 'r') as file:
        line = file.readline();
        while ";" in line:
            line = file.readline();
         
        positive_lexicon = file.readlines()    
    with open(os.path.join(os.path.abspath('opinion-lexicon-English/') , 'negative-words.txt'), 'r', encoding = "ISO-8859-1") as file:
        line = file.readline();
        while ";" in line:
            line = file.readline();
        
        negative_lexicon = file.readlines()
        
    positive_lexicon = list(map(lambda word: word.rstrip("\n\r"), positive_lexicon))
    negative_lexicon = list(map(lambda word: word.rstrip("\n\r"), negative_lexicon))
    
        
df = read_file('dataset/bing_liu/Nokia 6610.txt')
read_lexicon()

In [15]:
df.head()

,review,target
0,i am a business user who heavily depend on mob...,
1,there is much which has been said in other rev...,"phone[+3], work[+2]"
2,just double check with customer service to ens...,
3,after several years of torture in the hands of...,at&t customer service[-2]
4,"i have had the phone for 1 week , the signal q...",signal quality[+3]


In [16]:
import os
os.environ["CORENLP_HOME"] = r'C:\stanford-corenlp-full-2018-10-05'

import corenlp 
client = corenlp.CoreNLPClient()

def preprocess(sentences):
    res = []
    for sentence in sentences:
        text = sentence.lower()
        try:
            pattern = '[{pos:NN}][{pos:NN}][{pos:NN}] | [{pos:NN}][{pos:NN}]'
            matches = client.tokensregex(sentence, pattern)
            response = matches['sentences'][0]
            len_chunk = response['length']
            if len_chunk > 0:
                for index in range(0, len_chunk):
                    replacer = response[str(index)]['text']
                    text = text.replace(replacer, '-'.join(replacer.split(' ')))
        except:
            text = text
        res.append(text)
    return res

In [17]:
import requests

def get_tregex(text, tregex):
    url = "http://localhost:9001/tregex"
    request_params = {"pattern": tregex}
    r = requests.post(url, data=text, params=request_params, timeout=120)
    try:
        return r.json()['sentences'][0]
    except:
        return []

def sentence_from_tree(s):
    pattern = r'(?<= )[a-zA-Z].*?(?=\))'
    replaced = s.replace('\r\n', '')
    res = ' '.join(re.findall(pattern, replaced))
    return res
        
def sentence_type(clauses):
    IC = 0
    DC = 0
    for clause in clauses:
        if(clause[1] == 'IC'):
            IC += 1
        elif(clause[1] == 'DC'):
            DC += 1

    if IC == 1 and DC == 0:
        return 'simple_sentence'
    elif IC >= 2 and DC == 0:
        return 'compound_sentence'
    elif IC ==1 and DC >= 1:
        return 'complex_sentence'
    elif IC > 1 and DC >= 1:
        return 'compound_complex_sentence'
    else:
        return 'phrase'
    
def get_clauses(sentence):
    temp = []
    clauses = []
    
    res_all_clauses = get_tregex(sentence, 'S < (NP $ VP)') 
    res_sbar_clause = get_tregex(sentence, 'SBAR < S')
    #filter clauses with dependency clauses
    for x in range(0, len(res_all_clauses)):
        s = sentence_from_tree(res_all_clauses[str(x)]['match'])
        ic = True    
        for y in range(0, len(res_sbar_clause)):
            sbar = sentence_from_tree(res_sbar_clause[str(y)]['match'])            
            if sbar in s and sbar != s:
                s = s.replace(sbar, '')
                if(len(res_sbar_clause) == 1 and sbar != ''):
                    temp.append([sbar.strip(), 'DC'])
            elif s in sbar and sbar != '':
                ic = False
                temp.append( [sbar.strip(), 'DC'])
        if ic:
            temp.append( [s.strip(), 'IC'] )

    #overwrite sentence that already exist in list
    len_clause = len(temp)
    for x in range(0, len_clause):
        for y in range(x + 1, len_clause):
            temp[x][0] = temp[x][0].replace(temp[y][0], '').strip()
        
        temp[x][0] = re.sub(r"  ", " ", temp[x][0])
        if(temp[x][0] != ''):
            clauses.append( tuple(temp[x]) )
    #sorted by index sentence
    
    if(len(clauses) == 0):
        clauses.append((sentence, 'Phrase'))
    
    return sorted(clauses, key=lambda clause: 999 if sentence.find(clause[0]) == -1 else sentence.find(clause[0]))

In [18]:
from nltk.parse.corenlp import CoreNLPDependencyParser

parser = CoreNLPDependencyParser()

# adding new dependencies for DP 'nmod' and'advmod'
dep_DP = ['amod', 'prep', 'nsubj', 'csubj', 'xsubj', 'dobj', 'iobj']
conj_DP = ['conj']

In [19]:
def rule_1_1(parse, f, o_expanded):
    #Rule 1.1
    for (w1, dep, w2) in list(parse.triples()):
        if(dep in dep_DP):
            # Rule 1.1
            if(w1[0] in o_expanded):
                if w2[1] == 'NN' and w2[0] not in f:
                    return True, w2[0], w1[0]
            elif(w2[0] in o_expanded):
                if w1[1] == 'NN' and w1[0] not in f:
                    return True, w1[0], w2[0]
    return False, None, None

def rule_1_2(parse, f, o_expanded):
    # Rule 1.2
    for (w1, dep, w2) in parse.triples():
        if(dep in dep_DP):
            H = ''
            O = ''
            if w1[0] in o_expanded:
                H = w2[0]
                O = w1
            elif w2[0] in o_expanded:
                H = w1[0]
                O = w2

            if H:
                for (w1, dep, w2) in list(parse.triples()):
                    if w1[0] == H and w2[0] != O[0]:
                        if w2[1] == 'NN' and w2[0] not in f:
                            return True, w2[0], O[0]
                    elif w2[0] == H  and w1[0] != O[0]:
                        if w1[1] == 'NN' and w1[0] not in f:
                            return True, w1[0], O[0]
    return False, None, None

def rule_4_1(parse, f, o_expanded):
    # Rule 4.1
    for (w1, dep, w2) in list(parse.triples()):
        if(dep in conj_DP):
            if w1[0] in o_expanded:
                if w2[1] == 'JJ' and w2[0] not in o_expanded:
                    return True, w2[0]

            elif w2[0] in o_expanded:
                if w1[1] == 'JJ' and w1[0] not in o_expanded:
                    return True, w1[0]

    return False, None

def rule_4_2(parse, f, o_expanded):
    # Rule 4.2
    for (w1, dep, w2) in parse.triples():
        if(dep in dep_DP or dep in conj_DP):
            H = ''
            O = ''
            if w1[0] in o_expanded:
                H = w2[0]
                O = w1
            elif w2[0] in o_expanded:
                H = w1[0]
                O = w2

            if H:
                for (w1, dep, w2) in list(parse.triples()):
                    if w1[0] == H and w2[0] != O[0]:
                        if w2[1] == 'JJ' and w2[0] not in o_expanded:
                            return True, w2[0]

                    elif w2[0] == H  and w1[0] != O[0]:
                        if w1[1] == 'JJ' and w1[0] not in o_expanded:
                            return True, w1[0]
    
    return False, None

def rule_3_1(parse, f):
    #Rule 3.1
    for (w1, dep, w2) in list(parse.triples()):
        if(dep in conj_DP):
            if(w1[0] in f): 
                if w2[1] == 'NN' and w2[0] not in f:
                    return True, w2[0], w1[0]
            elif(w2[0] in f):          
                if w1[1] == 'NN' and w1[0] not in f:
                    return True, w1[0], w2[0]
                
    return False, None, None

def rule_3_2(parse, f):
    # Rule 3.2
    for (w1, dep, w2) in parse.triples():
        if(dep in dep_DP or dep in conj_DP):
            H = ''
            O = ''
            if w1[0] in f:
                H = w2[0]
                O = w1
            elif w2[0] in f:
                H = w1[0]
                O = w2

            if H:
                for (w1, dep, w2) in list(parse.triples()):
                    if w1[0] == H and w2[0] != O[0]:
                        if w2[1] == 'NN' and w2[0] not in f:
                            return True, w2[0], O[0]
                    elif w2[0] == H  and w1[0] != O[0]:
                        if w1[1] == 'NN' and w1[0] not in f:
                            return True, w1[0], O[0]

    return False, None, None

def rule_2_1(parse, f, o_expanded):
    # Rule 2.1
    for (w1, dep, w2) in list(parse.triples()):
        if(dep in dep_DP):
            if w1[0] in f:
                if w2[1] == 'JJ' and w2[0] not in o_expanded:
                    return True, w2[0]

            elif w2[0] in f:
                if w1[1] == 'JJ' and w1[0] not in o_expanded:
                    return True, w1[0]
                
    return False, None

def rule_2_2(parse, f, o_expanded):      
    # Rule 2.2
    for (w1, dep, w2) in parse.triples():
        if(dep in dep_DP):
            H = ''
            O = ''
            if w1[0] in f:
                H = w2[0]
                O = w1
            elif w2[0] in f:
                H = w1[0]
                O = w2

            if H:
                for (w1, dep, w2) in list(parse.triples()):
                    if w1[0] == H and w2[0] != O[0]:
                        if w2[1] == 'JJ' and w2[0] not in o_expanded:    
                            return True, w2[0]
                    elif w2[0] == H  and w1[0] != O[0]:
                        if w1[1] == 'JJ' and w1[0] not in o_expanded:
                            return True, w1[0]

    return False, None

def clause(other_clause, temp, parse, f):
    res = []
    if other_clause and len(temp) == 0:
        for (w1, dep, w2) in list(parse.triples()):
            if w1[1] == 'NN' and w1[0] not in f:
                res.append(w1[0])
    if len(res) > 0:
        return True, res
    return False, None

In [20]:
import pandas as pd
candidate_aspect = []
new_opinion = []
op_set = positive_lexicon + negative_lexicon

def double_propagation(O: op_set, reviews, targets, output_file, save_to_file = False, using_clause = False):
    o_expanded = O
    f = []
    is_stop = False
    flag_cycle = 0
    
    t_a_p = []
    pair_a_o = []
    a_p = []
    r_p = []
    
    stop_a = 0
    stop_o = 0
    
    while (not is_stop):    
        index = 0
        stop_a = len(f)
        stop_o = len(o_expanded)

        for sentence in reviews:
            clauses = []
            if using_clause and flag_cycle == 0:
                temp = get_clauses(sentence)
                length = len(temp)
                
                if length == 0:
                    clauses.append((sentence, ''))
                else:
                    for x in range(length - 1, -1, -1):
                        clauses.append(temp[x])
            else:
                clauses.append((sentence, ''))
            
            temp = []
            other_clause = False
            pair = []

            for r in clauses:
                temp2 = []

                try:
                    parse = next(parser.raw_parse(r[0]))
                except:
                    continue

                aspect_sentence = []
                if flag_cycle:
                    aspect_sentence = t_a_p[index].split('|')

                # Rule 1.1
                is_true, t_a, o = rule_1_1(parse, aspect_sentence, o_expanded)
                if is_true:
                    f.append(t_a)
                    temp.append(t_a)
                    temp2.append(t_a)
                    pair.append(t_a + '!' + o)

                # Rule 1.2
                is_true, t_a, o = rule_1_2(parse, aspect_sentence, o_expanded)
                if is_true:
                    f.append(t_a)
                    temp.append(t_a)
                    temp2.append(t_a)
                    pair.append(t_a + '!' + o)

                # Rule 4.1
                is_true, t_o = rule_4_1(parse, f, o_expanded)
                if is_true:
                    o_expanded.append(t_o)

                # Rule 4.2
                is_true, t_o = rule_4_2(parse, f, o_expanded)
                if is_true:
                    o_expanded.append(t_o)

                # Rule 3.1
                is_true, t_a, o = rule_3_1(parse, aspect_sentence)
                if is_true:
                    f.append(t_a)
                    temp.append(t_a)
                    temp2.append(t_a)
                    pair.append(t_a + '!' + o)

                # Rule 3.2
                is_true, t_a, o = rule_3_2(parse, aspect_sentence)
                if is_true:
                    f.append(t_a)
                    temp.append(t_a)
                    temp2.append(t_a)
                    pair.append(t_a + '!' + o)

                # Rule 2.1
                is_true, t_o = rule_2_1(parse, f, o_expanded)
                if is_true:
                    o_expanded.append(t_o)

                # Rule 2.2
                is_true, t_o = rule_2_2(parse, f, o_expanded)
                if is_true:
                    o_expanded.append(t_o)
                    
                if using_clause and flag_cycle == 0:
                    flag_o = False
                    if r[1] == 'DC':
                        for word in r[0].split(' '):
                            if word in o_expanded:
                                flag_o = True
                                break

                    # clause
                    is_true, t_a = clause(other_clause, temp, parse, aspect_sentence)
                    if is_true:
                        for x in t_a:
                            f.append(x)
                            temp.append(x)

                    if flag_o and len(temp2) == 0:
                        other_clause = True
            
            if flag_cycle == 0:

                r_p.append(sentence)
                a_array = []
                tes = []
                
                try:
                    tes = targets[index].split(', ')
                except:
                    tes = []
                for x in tes:
                    splitted = x.split('[')
                    if len(splitted) > 1:
                        a_array.append(splitted[0] + '!' + splitted[1][0])
                    else:
                        a_array.append(splitted[0])

                a_p.append('|'.join(a_array))
                t_a_p.append('|'.join(temp))
                pair_a_o.append('|'.join(pair))
            else:
                if len(temp) != 0:
                    t_a_p[index] += '|' + '|'.join(temp)
                    pair_a_o[index] += '|' + '|'.join(pair)
            index += 1

        flag_cycle = 1
        
        if(len(f) == stop_a and len(o_expanded) == stop_o):
            if save_to_file == True:
                data = { 'review':r_p, 'aspect': a_p, 'prediction': pair_a_o}
                out = pd.DataFrame(data)
                out.to_csv(output_file)
            is_stop = True
        
    return f, o_expanded

In [8]:
dp_aspect, opinion_expand = double_propagation(op_set, df['review'], df['target'], 'nokia.csv', True, True)

In [9]:
def calculate_frequency(aspects):
    aspect_frequency = {}
    
    for aspect in aspects:
        if(aspect in aspect_frequency):
            aspect_frequency[aspect] += 1
        else:
            aspect_frequency[aspect] = 1
            
    return aspect_frequency

k_DP = ['conj', 'compound']
def pruning_based_on_clause(aspect_frequency, reviews, predictions):
    pruning = []
    index = 0
    for review in reviews:
        temp = predictions[index]
        index += 1
        if isinstance(temp, str):
            prediction = []
            for t in temp.split('|'):
                prediction.append(t.split('!')[0])
        else:
            continue
        parse = next(parser.raw_parse(review))
        for (w1, dep, w2) in list(parse.triples()):
            if(w1[0] in aspect_frequency and w2[0] in aspect_frequency):
                if(dep not in k_DP):
                    if(aspect_frequency[w1[0]] > aspect_frequency[w2[0]]):
                        if w2[0] in prediction:
                            pruning.append(w2[0])
                    elif(aspect_frequency[w1[0]] < aspect_frequency[w2[0]]):
                        if w1[0] in prediction:
                            pruning.append(w1[0])
                
    return pruning
def pruning_based_other_products_and_dealers(aspect_frequency, reviews,predictions, window=3):
    pruning = []
    ProductINDI = ["compare to", "compare with", "better than", "worse than"]
    DealerINDI  = ["shop with", "buy from"]
    count = 0
    for review in reviews:
        temp = predictions[count]
        count += 1
        if isinstance(temp, str):
            prediction = []
            for t in temp.split('|'):
                prediction.append(t.split('!')[0])
        else:
            continue
        if any(indication in review for indication in ProductINDI):
            tokens = nltk.word_tokenize(review)
            index = 0
            while index < len(tokens) - 1:
                if tokens[index] + " " + tokens[index + 1] in ProductINDI:
                    index += 2
                    for x in range(window):
                        next_window = index + x + 1;
                        if next_window < len(tokens) and tokens[next_window] in aspect_frequency:
                            if tokens[next_window] in prediction:
                                pruning.append(tokens[next_window])
                else :
                    index += 1
                    
        if any(indication in review for indication in DealerINDI):
            tokens = nltk.word_tokenize(review)
            index = 0
            while index < len(tokens) - 1:
                if tokens[index] + " " + tokens[index + 1] in DealerINDI:
                    index += 2
                    for x in range(window):
                        next_window = index + x + 1;
                        if next_window < len(tokens) and tokens[next_window] in aspect_frequency:
                            if tokens[next_window] in prediction:
                                pruning.append(tokens[next_window])
                else :
                    index += 1 
                        
    return pruning

In [10]:
import pandas as pd

hs = pd.read_csv("nokia.csv")

import re
import math
import numpy as np
target = []

for t in hs['aspect']:
    if t is not np.nan:
        for s in t.split('|'):
            target.append(s.split('!')[0])
            
tp = calculate_frequency(target)
ap = calculate_frequency(dp_aspect)
pruning_clause = pruning_based_on_clause(ap,hs['review'], hs['prediction'])
prun_dealer_product = pruning_based_other_products_and_dealers(ap, hs['review'], hs['prediction'])

a = pruning_clause + prun_dealer_product

for x in list(dict.fromkeys(dp_aspect)):
    flag = True
    splitted = x.split('-')
    for word in splitted:
        for match in list(dict.fromkeys(dp_aspect)):
            if word in match and word != match:
                flag = False
                
    if ap[x] == 1 and flag:
        ap[x] -= 1

for freq in list(dict.fromkeys(a)):
    try:
        if ap[freq] > 0:
            ap[freq] -= 1
    except: continue

new_dp_aspect = []
for a in ap:
    if ap[a] > 0:
        new_dp_aspect.append(a)

In [11]:
def calculate_performance():
    atas_recall = 0
    atas_precision = 0
    bawah_precision = 0
    bawah_recall = 0

    new_target = []
    for x in list(dict.fromkeys(target)):
        for y in x.split(' '):
            new_target.append(y.split('!')[0])

    new_aspect = []
    for x in list(dict.fromkeys(new_dp_aspect)):
        for y in x.split('-'):
            new_aspect.append(y.split('!')[0])

    for x in tp:
        bawah_precision += tp[x]
        for y in x.split(' '):
            flag = False
            for apt in new_aspect:
                if y in apt:
                    atas_precision += tp[x]
                    flag = True
                    break
            if flag:
                break

    for x in ap:
        bawah_recall += ap[x]
    #     print(x.split('-'))
        for y in x.split('-'):
            flag = False
            for tgt in new_target:
                if y in tgt:
        #             print(y)
                    atas_recall += ap[x]
                    flag = True
                    break
            if flag:
                break

    precision = atas_precision/bawah_precision
    recall = atas_recall/bawah_recall
    f1 = (2 * precision * recall) / (precision + recall)
    
    return precision, recall, f1

In [12]:
calculate_performance() 

(0.8461538461538461, 0.7162629757785467, 0.7758091993185691)

In [13]:
calculate_performance() 

In [85]:
dict.fromkeys(target)

{'': None,
 '4mp': None,
 '4mp camera': None,
 '4mp resolution': None,
 'auto mode': None,
 'auto setting': None,
 'automode': None,
 'battery': None,
 'battery charging system': None,
 'battery life': None,
 'body': None,
 'camera': None,
 'canera': None,
 'canon': None,
 'canon g3': None,
 'canon powershot g3': None,
 'casing': None,
 'color': None,
 'compactflash': None,
 'control': None,
 'darn diopter adjustment dial': None,
 'delay': None,
 'depth': None,
 'design': None,
 'dial': None,
 'digital camera': None,
 'display': None,
 'distortion': None,
 'download': None,
 'exposure control': None,
 'external flash hot shoe': None,
 'feature': None,
 'feel': None,
 'finish': None,
 'flash': None,
 'flash photo': None,
 'focus': None,
 'four megapixel': None,
 'function': None,
 'g3': None,
 'grain': None,
 'highlight': None,
 'hot shoe flash': None,
 'image': None,
 'image quality': None,
 'import': None,
 'lag': None,
 'lag time': None,
 'lcd': None,
 'learning': None,
 'lens': None

In [86]:
dict.fromkeys(new_dp_aspect)

{'4x': None,
 'ability': None,
 'adapter': None,
 'alsmost': None,
 'anyone': None,
 'anything': None,
 'auto': None,
 'b': None,
 'balance': None,
 'battery': None,
 'bit': None,
 'box': None,
 'brand': None,
 'buck': None,
 'button': None,
 'buy': None,
 'camera': None,
 'canon': None,
 'cap': None,
 'card': None,
 'charge': None,
 'choice': None,
 'color': None,
 'combination': None,
 'compactflash': None,
 'compromise': None,
 'computer': None,
 'consumer': None,
 'control': None,
 'coolpix': None,
 'cord': None,
 'corner': None,
 'cost': None,
 'couple': None,
 'day': None,
 'deal': None,
 'depth': None,
 'design': None,
 'detail': None,
 'dial': None,
 'difference': None,
 'digicam': None,
 'disappointment': None,
 'drive': None,
 'effect': None,
 'enthusiast': None,
 'experience': None,
 'exposure': None,
 'fact': None,
 'feel': None,
 'fiance': None,
 'film': None,
 'finish': None,
 'flash': None,
 'flaw': None,
 'flexibility': None,
 'focus': None,
 'format': None,
 'frame': N